# Рекомендация тарифов

По данным оператора мобильной связи многие клиенты пользуются архивными тарифами. Компания планирует построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра». Иммется информация о поведении клиентов, которые уже перешли на эти тарифы (из проекта "Определение перспективного тарифа для телеком компании").

Необходимо построить модель для задачи классификации, которая выберет подходящий тариф с максимально большим значением accuracy. Долю правильных ответов нужно довести до 0.75 и проверить accuracy на тестовой выборке.

## Открытие и изучение файла

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.info()
display(df.head())
features = df.drop('is_ultra',axis=1) # создаем переменную с признаками
target = df['is_ultra']  # создаем переменную с  целевым признаком

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


## Деление данных на выборки

Разбиваем данные на обучающую, валидационную и тестовую выборки в соотношении 60-20-20.

In [3]:
features_t, features_valid, target_t, target_valid = train_test_split(
    features, target, test_size=0.2,train_size=0.8, random_state=42)
features_train, features_test, target_train, target_test = train_test_split(
    features_t, target_t, test_size=0.25,train_size=0.75, random_state=42)

## Исследование моделей

На данном этапе рассмотрим 3 алгоритма классификации:
- Решающее дерево
- Случайный лес
- Логистическая регрессия

Определим наилучший алгоритм.

### Применим алгоритм классификации "Решающее дерево", последовательно изменяя его гиперпараметры с целью нахождения максимального значения Accuracy.

#### В алгоритме DecisionTreeClassifier найдем глубину дерева, при котором будет достигнуто максимальное значение Accuracy

In [4]:
best_model = None
best_result = 0
for depth in range(1,6):
    model = DecisionTreeClassifier(random_state=42, max_depth=depth)
    model.fit(features_train,target_train)
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid,predictions_valid)
    print("Max depth=",depth,": ",result)
    if result > best_result:
        best_model = model# сохраните наилучшую модель
        best_result = result#  сохраните наилучшее значение метрики accuracy на валидационных данных
print("Accuracy наилучшей модели на валидационной выборке:", best_result)

Max depth= 1 :  0.7698289269051322
Max depth= 2 :  0.7900466562986003
Max depth= 3 :  0.7776049766718507
Max depth= 4 :  0.7900466562986003
Max depth= 5 :  0.8040435458786936
Accuracy наилучшей модели на валидационной выборке: 0.8040435458786936


Наибольшее значение Accuracy достигается при глубине дерева Max depth= 5 
#### В алгоритме DecisionTreeClassifier найдем параметр min_samples_leaf, при котором будет достигнуто максимальное значение Accuracy

In [5]:
best_model = None
best_result = 0
for i in range(1,6):
    model = DecisionTreeClassifier(random_state=42, max_depth=5, min_samples_leaf=i)
    model.fit(features_train,target_train)
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid,predictions_valid)
    print("min_samples_leaf=",i,": ",result)
    if result > best_result:
        best_model = model# сохраните наилучшую модель
        best_result = result#  сохраните наилучшее значение метрики accuracy на валидационных данных
print("Accuracy наилучшей модели на валидационной выборке:", best_result)

min_samples_leaf= 1 :  0.8040435458786936
min_samples_leaf= 2 :  0.80248833592535
min_samples_leaf= 3 :  0.8040435458786936
min_samples_leaf= 4 :  0.8009331259720062
min_samples_leaf= 5 :  0.80248833592535
Accuracy наилучшей модели на валидационной выборке: 0.8040435458786936


Наибольшее значение Accuracy достигается при параметре min_samples_leaf= 1
#### В алгоритме DecisionTreeClassifier найдем параметр min_samples_split, при котором будет достигнуто максимальное значение Accuracy

In [6]:
best_model = None
best_result = 0
for j in range(2,6):
    model = DecisionTreeClassifier(random_state=42, max_depth=5, min_samples_leaf=1, min_samples_split=j)
    model.fit(features_train,target_train)
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid,predictions_valid)
    print("min_samples_split=",j,": ",result)
    if result > best_result:
        best_model = model# сохраните наилучшую модель
        best_result = result#  сохраните наилучшее значение метрики accuracy на валидационных данных
print("Accuracy наилучшей модели на валидационной выборке:", best_result)

min_samples_split= 2 :  0.8040435458786936
min_samples_split= 3 :  0.8040435458786936
min_samples_split= 4 :  0.8040435458786936
min_samples_split= 5 :  0.8040435458786936
Accuracy наилучшей модели на валидационной выборке: 0.8040435458786936


При изменении параметра min_samples_split значение Accuracy постоянно.

### Применим алгоритм классификации "Случайный лес" и определим гиперпараметр n_estimators, при котором значение Accuracy будет максимальным

In [7]:
best_model = None
best_result = 0
for est in range(10,14,1):
    model = RandomForestClassifier(random_state=42, n_estimators=est) # обучаем модель с заданным количеством деревьев
    model.fit(features_train,target_train) # обучаем модель на тренировочной выборке
    result = model.score(features_valid,target_valid) # посчитайте качество модели на валидационной выборке
    print("n_estimators=",est,": ",result)
    if result > best_result:
        best_model = model#  наилучшая модель
        best_result = result#  наилучшее значение accuracy на валидационных данных

print("Accuracy наилучшей модели на валидационной выборке:", best_result)

n_estimators= 10 :  0.8087091757387247
n_estimators= 11 :  0.7962674961119751
n_estimators= 12 :  0.807153965785381
n_estimators= 13 :  0.7993779160186625
Accuracy наилучшей модели на валидационной выборке: 0.8087091757387247


Наибольшее значение Accuracy достигается при параметре min_samples_leaf= 10

### Применим алгоритм классификации "Логистическая регрессия" и определим гиперпараметр max_iter, при котором значение Accuracy будет максимальным

In [8]:
best_model = None
best_result = 0
for iter in range(100, 1000,100):
    model = LogisticRegression(random_state=42, solver='lbfgs', max_iter=iter)
    model.fit(features_train,target_train) #  модель на тренировочной выборке
    result = model.score(features_valid,target_valid)
    print("max_iter=",iter,": ",result)
    if result > best_result:
        best_model = model# наилучшая модель
        best_result = result#  наилучшее значение accuracy на валидационных данных

print("Accuracy наилучшей модели на валидационной выборке:", best_result)

max_iter= 100 :  0.702954898911353
max_iter= 200 :  0.702954898911353
max_iter= 300 :  0.702954898911353
max_iter= 400 :  0.702954898911353
max_iter= 500 :  0.702954898911353
max_iter= 600 :  0.702954898911353
max_iter= 700 :  0.702954898911353
max_iter= 800 :  0.702954898911353
max_iter= 900 :  0.702954898911353
Accuracy наилучшей модели на валидационной выборке: 0.702954898911353


**Вывод:**

Таким образом мы определили, что максимальный Accuracy = 0.8087091757387247 по алгоритму "Случайный лес" с гиперпараметрами:
random_state=42, n_estimators=10
Используем этот алгоритм классификации с указанными гиперпараметрами для проверки модели на тестовой выборке.

## Проверка модель на тестовой выборке

In [9]:
model = RandomForestClassifier(random_state=42, n_estimators=10)
model.fit(features_train,target_train)
predictions_test = model.predict(features_test)
result = model.score(features_test,target_test)
result 

0.7667185069984448

Доля правильных ответов более 0,75. Цель достигнута

## (бонус) Проверка модели на адекватность

Проверку на адекватность модели реализуем с помощью DummyClassifier

In [10]:
model = DummyClassifier(strategy='most_frequent', random_state=42)
model.fit(features_train,target_train)
result = model.score(features_test,target_test)
result

0.7013996889580093

Качество полученной модели ниже, значит наша модель адекватна.